# Splitsville: Fun with the New **regex** Module

## Rex Dwyer
## TriPython Monthly Meeting
## April 28, 2016

The Python regular expression module **re** has a number of deficiencies.  There are several cases where its behavior differs from Perl, the gold standard for regular expressions. The **regex** module is intended to replace **re**. **regex** can be found at https://pypi.python.org/pypi/regex/2016.04.25 In addition to fixing some bugs, it adds a lot of new features. This notebook reviews a few 'advanced' features of regular expressions, then introduces some of the new features of **regex**.  It's not intended to be comprehensive.

First, let's review a few things from the old **re** module that haven't changed.

In [1]:

from re import *

t = 'The quick brown fox born on 1/23/2013 jumped over the lazy dog born on 10/6/10.'


### match and search
Use **r'...'** (raw strings) for patterns, because we like to use elements like **\w**, and we don't want to have to type **\\\\w**.
**match** looks for the pattern at the very beginning of the target string.


In [2]:

match(r'The',t)


<_sre.SRE_Match object; span=(0, 3), match='The'>

(Normally if we were going to reuse a pattern, we'd compile it and use the **match** method of the resulting pattern like this:

In [3]:
p = compile(r'The')
p.match(t)

<_sre.SRE_Match object; span=(0, 3), match='The'>

But in this tutorial, we will not compile.)

This one doesn't match, because **the** is not at the beginning of the target string.  It returns **None**, which of course doesn't print any output.

In [4]:

match(r'the',t)


**search** looks at the whole string.  It finds the **the** before **lazy dog**.

In [5]:

search(r'the',t)


<_sre.SRE_Match object; span=(50, 53), match='the'>

**search** and **match** can be made case-insensitive.

In [6]:

search(r'the',t, IGNORECASE)


<_sre.SRE_Match object; span=(0, 3), match='The'>

In [7]:

search(r'(?i)the',t)


<_sre.SRE_Match object; span=(0, 3), match='The'>

### **findall** and **finditer**
**findall** lists every match to the pattern, but doesn't give the position.  **I** is short for **IGNORECASE**.

In [8]:

findall(r'the',t, I)


['The', 'the']

**finditer** returns an iterator over all matches of the pattern in the string.

In [9]:

finditer(r'the',t, I)


In [10]:

list(finditer(r'the',t, I))


[<_sre.SRE_Match object; span=(0, 3), match='The'>,
 <_sre.SRE_Match object; span=(50, 53), match='the'>]

In [11]:

list(finditer(r'(brown|lazy) (dog|fox)',t))


[<_sre.SRE_Match object; span=(10, 19), match='brown fox'>,
 <_sre.SRE_Match object; span=(54, 62), match='lazy dog'>]

The **X** option lets us add spaces to the pattern for readability -- but in this case, we wanted to match the space between the two words, so we need to escape it with **\\**.

In [12]:

list(finditer(r'(brown | lazy) (dog | fox)', t, X))


[]

In [13]:

list(finditer( r'(brown | lazy)\ (dog | fox)', t, X ))


[<_sre.SRE_Match object; span=(10, 19), match='brown fox'>,
 <_sre.SRE_Match object; span=(54, 62), match='lazy dog'>]

Now let's find all the words with an internal **o**.  **\\w+** matches one or more 'word' characters: letters, digits, and underscore.

In [14]:

findall(r'\w+o\w+', t)


['brown', 'fox', 'born', 'dog', 'born']

This one also matches words that begin or end with **o**, because **\\w\*** matches zero or more characters.

In [15]:

findall(r'\w*o\w*', t)


['brown', 'fox', 'born', 'on', 'over', 'dog', 'born', 'on']

### Substitution with **sub**
Now a couple of substitutions... Note that (unlike in Perl) substitution returns a new string.  That's no surprise, because Python strings are immutable.

In [16]:

sub(r'\w*o\w*', 'o-word', t)


'The quick o-word o-word o-word o-word 1/23/2013 jumped o-word the lazy o-word o-word o-word 10/6/10.'

In [17]:

t


'The quick brown fox born on 1/23/2013 jumped over the lazy dog born on 10/6/10.'

### Capturing parts of the pattern match
We can capture things with parentheses, then refer to the captured items by **\1, \2**, etc. in the substitution.

In [18]:

sub(r'(\w*)o(\w*)', r'=\1o\2=', t)


'The quick =brown= =fox= =born= =on= 1/23/2013 jumped =over= the lazy =dog= =born= =on= 10/6/10.'

We can also give things names when we capture them.

In [19]:

sub(r'(?P<before>\w*)o(?P<after>\w*)', r'=\g<before>o\g<after>=', t)


'The quick =brown= =fox= =born= =on= 1/23/2013 jumped =over= the lazy =dog= =born= =on= 10/6/10.'

We can make the pattern, but apparently not the substitution, clearer by adding (?x) and spaces.

In [20]:

sub(r'(?x) (?P<before> \w*) o (?P<after> \w*)', r'=\g<before>o\g<after>=', t)


'The quick =brown= =fox= =born= =on= 1/23/2013 jumped =over= the lazy =dog= =born= =on= 10/6/10.'

Yes, this would have been easier:

In [21]:

sub(r'(?x) (\w*o\w*)', r'=\1=', t)


'The quick =brown= =fox= =born= =on= 1/23/2013 jumped =over= the lazy =dog= =born= =on= 10/6/10.'

### Greedy and lazy matching
Let's try to get the "day" portion of each date. It is the first and only captured group in this pattern.

In [22]:

fi = finditer(r'(?x) / (.* ) /', t)

[m.group(1) for m in fi]


['23/2013 jumped over the lazy dog born on 10/6']

What went wrong? By default, **\*** is greedy.  We can make it lazy by adding **?**.

In [23]:

fi = finditer(r'(?x) / (.*? ) /', t)

[m.group(1) for m in fi]


['23', '6']

This works for the other quantifiers as well.  (**+, ?, {m,n}**)

This one is eager and has an internal **e** in the first match and an internal **o** in the second match.

In [24]:

fi = finditer(r'(?x) ([aeiou]) .* \1', t)

[m.group(0) for m in fi]


['e quick brown fox born on 1/23/2013 jumped over the', 'og born o']

This one is lazy.

In [25]:

fi = finditer(r'(?x) ([aeiou]) .*? \1', t)

[m.group(0) for m in fi]


['e quick brown fox born on 1/23/2013 jumpe', 'over the lazy do', 'orn o']

### split
**split** works differently in **regex**.  Let's start with **re**.
This is a familiar split on whitespace.

In [26]:

split(r'\s+', t)


['The',
 'quick',
 'brown',
 'fox',
 'born',
 'on',
 '1/23/2013',
 'jumped',
 'over',
 'the',
 'lazy',
 'dog',
 'born',
 'on',
 '10/6/10.']

And this splits on the letter **o**:

In [27]:

split(r'o', t)


['The quick br',
 'wn f',
 'x b',
 'rn ',
 'n 1/23/2013 jumped ',
 'ver the lazy d',
 'g b',
 'rn ',
 'n 10/6/10.']

Here's a gotcha:

In [28]:

split(r'o', t, IGNORECASE)


['The quick br',
 'wn f',
 'x born on 1/23/2013 jumped over the lazy dog born on 10/6/10.']

The third argument to **split** does not give options.  It gives the maximum number of splits.  We unknowingly specified 2:

In [29]:

IGNORECASE


2

For split, it is better to use **(?i)** for case-insensitivity.

In [30]:

split(r'(?i)the', t)


['',
 ' quick brown fox born on 1/23/2013 jumped over ',
 ' lazy dog born on 10/6/10.']

Or you can use the **flags** keyword argument.

In [31]:

split(r'the', t, flags=IGNORECASE)


['',
 ' quick brown fox born on 1/23/2013 jumped over ',
 ' lazy dog born on 10/6/10.']

### Capturing in the pattern for **split**

This one splits on words with an internal **u**:

In [32]:

split(r'\w+u\w+', t)


['The ',
 ' brown fox born on 1/23/2013 ',
 ' over the lazy dog born on 10/6/10.']

But what if we wanted to save the **u**-words?  We can use parentheses to capture things inside the splitting pattern!

In [33]:

split(r'(\w+u\w+)', t)


['The ',
 'quick',
 ' brown fox born on 1/23/2013 ',
 'jumped',
 ' over the lazy dog born on 10/6/10.']

In [34]:

split(r'\w*o\w*', t)


['The quick ',
 ' ',
 ' ',
 ' ',
 ' 1/23/2013 jumped ',
 ' the lazy ',
 ' ',
 ' ',
 ' 10/6/10.']

In [35]:

split(r'(\w*o\w*)', t)


['The quick ',
 'brown',
 ' ',
 'fox',
 ' ',
 'born',
 ' ',
 'on',
 ' 1/23/2013 jumped ',
 'over',
 ' the lazy ',
 'dog',
 ' ',
 'born',
 ' ',
 'on',
 ' 10/6/10.']

Here we split on 4-letter words.  Then we do it again and capture the 4-letter words.

In [36]:

split(r'\b\w{4}\b', t)


['The quick brown fox ',
 ' on 1/23/',
 ' jumped ',
 ' the ',
 ' dog ',
 ' on 10/6/10.']

In [37]:

split(r'\b(\w{4})\b', t)


['The quick brown fox ',
 'born',
 ' on 1/23/',
 '2013',
 ' jumped ',
 'over',
 ' the ',
 'lazy',
 ' dog ',
 'born',
 ' on 10/6/10.']

### **split** and zero-width assertions.
Here we split on word boundaries.  **\\b** is a zero-width assertion.  It requires that certain characters be present, but it doesn't "consume" them.

In [38]:

split(r'\b',t)


['The quick brown fox born on 1/23/2013 jumped over the lazy dog born on 10/6/10.']

So what went wrong?  Unlike **split** in Perl, the split function in **re** will not split on zero-width assertions. The new **regex** module gets this right.

In [39]:

from regex import *
split(r'\b',t)


['The quick brown fox born on 1/23/2013 jumped over the lazy dog born on 10/6/10.']

Oops. To get the new behavior, we must add the "Version 1" option to the regular expression.  "Version 0" emulates **re**.

In [40]:
split(r'(?V1)\b',t)

['',
 'The',
 ' ',
 'quick',
 ' ',
 'brown',
 ' ',
 'fox',
 ' ',
 'born',
 ' ',
 'on',
 ' ',
 '1',
 '/',
 '23',
 '/',
 '2013',
 ' ',
 'jumped',
 ' ',
 'over',
 ' ',
 'the',
 ' ',
 'lazy',
 ' ',
 'dog',
 ' ',
 'born',
 ' ',
 'on',
 ' ',
 '10',
 '/',
 '6',
 '/',
 '10',
 '.']

We can make life a little easier by setting the version globally.

In [41]:
import regex
regex.DEFAULT_VERSION = VERSION1
split(r'\b',t)

['',
 'The',
 ' ',
 'quick',
 ' ',
 'brown',
 ' ',
 'fox',
 ' ',
 'born',
 ' ',
 'on',
 ' ',
 '1',
 '/',
 '23',
 '/',
 '2013',
 ' ',
 'jumped',
 ' ',
 'over',
 ' ',
 'the',
 ' ',
 'lazy',
 ' ',
 'dog',
 ' ',
 'born',
 ' ',
 'on',
 ' ',
 '10',
 '/',
 '6',
 '/',
 '10',
 '.']

**\\m** and **\\M** are zero-width assertions that are true at the beginnings and ends of words.

In [42]:

split(r'\M',t)


['The',
 ' quick',
 ' brown',
 ' fox',
 ' born',
 ' on',
 ' 1',
 '/23',
 '/2013',
 ' jumped',
 ' over',
 ' the',
 ' lazy',
 ' dog',
 ' born',
 ' on',
 ' 10',
 '/6',
 '/10',
 '.']

In [43]:

split(r'\m',t)


['',
 'The ',
 'quick ',
 'brown ',
 'fox ',
 'born ',
 'on ',
 '1/',
 '23/',
 '2013 ',
 'jumped ',
 'over ',
 'the ',
 'lazy ',
 'dog ',
 'born ',
 'on ',
 '10/',
 '6/',
 '10.']

### Look-arounds
We can split on any 4-letter word.

In [44]:

split(r'(?x) \b \w{4} \b', t)


['The quick brown fox ',
 ' on 1/23/',
 ' jumped ',
 ' the ',
 ' dog ',
 ' on 10/6/10.']

But what if we want to split on any 4-letter word but **born**? We can use a look-ahead assertion.  Look-aheads and look-behinds come in two flavors: positive and negative.  All four are zero-width assertions.  They required certain characters to be present or absent, but don't consume the characters.  In this case, we need a negative assertion.  We could do a look-ahead:

In [45]:

split(r'(?x) \b (?!born) \w{4} \b', t)


['The quick brown fox born on 1/23/',
 ' jumped ',
 ' the ',
 ' dog born on 10/6/10.']

or a look-behind:

In [46]:

split(r'(?x) \b \w{4} (?<!born) \b', t)


['The quick brown fox born on 1/23/',
 ' jumped ',
 ' the ',
 ' dog born on 10/6/10.']

or, if we are feeling perverse, both:

In [47]:

split(r'(?x) \b (?!bor) \w{4} (?<!orn) \b', t)


['The quick brown fox born on 1/23/',
 ' jumped ',
 ' the ',
 ' dog born on 10/6/10.']

This one splits on any 4-letter word that doesn't contain **o**:

In [48]:

split(r'(?x) \b (?!\w*o) \w{4} \b', t)


['The quick brown fox born on 1/23/',
 ' jumped over the ',
 ' dog born on 10/6/10.']

This one splits on the letter **o**.  The **o** is consumed and lost.

In [49]:

split(r'(?x) o', t)


['The quick br',
 'wn f',
 'x b',
 'rn ',
 'n 1/23/2013 jumped ',
 'ver the lazy d',
 'g b',
 'rn ',
 'n 10/6/10.']

This one has a positive look-ahead assertion.  It splits before every **o**.

In [50]:

split(r'(?x) (?=o)', t)


['The quick br',
 'own f',
 'ox b',
 'orn ',
 'on 1/23/2013 jumped ',
 'over the lazy d',
 'og b',
 'orn ',
 'on 10/6/10.']

This one has a positive look-behind assertion.  It splits after evey **o**.

In [51]:

split(r'(?x) (?<=o)', t)


['The quick bro',
 'wn fo',
 'x bo',
 'rn o',
 'n 1/23/2013 jumped o',
 'ver the lazy do',
 'g bo',
 'rn o',
 'n 10/6/10.']

This one splits between  **o** and **r**:

In [52]:

split(r'(?x) (?<=o) (?=r)', t)


['The quick brown fox bo',
 'rn on 1/23/2013 jumped over the lazy dog bo',
 'rn on 10/6/10.']

The assertions could appear in either order.

In [53]:

split(r'(?x) (?=r) (?<=o)', t)


['The quick brown fox bo',
 'rn on 1/23/2013 jumped over the lazy dog bo',
 'rn on 10/6/10.']

This one splits between any two consecutive vowels.

In [54]:

split(r'(?x) (?<=[aeiou]) (?=[aeiou])', t)


['The qu',
 'ick brown fox born on 1/23/2013 jumped over the lazy dog born on 10/6/10.']

### Fun with DNA: Open reading frames
DNA is a sequence of bases, A, C, G, or T.  They are translated into proteins 3 bases at a time.  Each 3-base sequence is called a **codon**.  There is a special **start codon** ATG, and three **stop codons**, TGA, TAG, and TAA.  The start and stop codons are highlighted below.

In [55]:

dna = 'cgcgcATGcgcgcgTGAcgcgcgTAGcgcgcgcgc'

dna = dna.lower()


An **opening reading frame** or **ORF** consists of a start codon, followed by some more codons, and ending with a stop codon.  (In real life, "some more" is usually hundreds or thousands.)

In [56]:

orfpat = r'(?x) atg (...)* (tga|tag|taa)'

search(orfpat,dna)


<regex.Match object; span=(5, 26), match='atgcgcgcgtgacgcgcgtag'>

Actually, that's not quite right.  The internal codons should not be stop codons. We can handle that with a negative lookahead assertion.  (Can you think of another way?)

In [57]:

orfpat = r'(?x) atg  ( (?!tga|tag|taa) ... )*  (tga|tag|taa)'

search(orfpat,dna)


<regex.Match object; span=(5, 17), match='atgcgcgcgtga'>

We don't really want to capture the "some more codons" separately.  In a minute that will get in the way.  So we can use **(?:...)** to group without capturing:

In [58]:

orfpat = r'(?x) ( atg  (?: (?!tga|tag|taa) ... )*  (?:tga|tag|taa) )'

findall(orfpat,dna)


['atgcgcgcgtga']

Here is another DNA sequence.  Note that this one has overlapping ORFs.  We would like a list of **all** orfs, specifically **ATGcATGcgTGA** and **ATGcgTGAcTAA**.  Our last pattern only finds the first ORF. Since it consumes the first ORF, it also consumes the beginning of the second ORF.

In [59]:

dna = 'cgcgcATGcATGcgTGAcTAAcgTAGcgcgcgcgc'

dna = dna.lower()

findall(orfpat,dna)


['atgcatgcgtga']

Since we want to find something without consuming it, we can use a positive lookahead assertion.  We put the whole ORF pattern inside the lookahead.  We need to capture what is matched by the lookahead without consuming it.

In [60]:

orfpat = r'(?x) (?= ( atg  (?: (?!tga|tag|taa) ... )*  (?:tga|tag|taa) ))'

findall(orfpat,dna)


['atgcatgcgtga', 'atgcgtgactaa']

Notice the position of the capturing parentheses.  This doesn't work:

In [61]:

orfpat = r'(?x) ( (?= atg  (?: (?!tga|tag|taa) ... )*  (?:tga|tag|taa) ))'

findall(orfpat,dna)


['', '']

Why? Because the look-ahead assertion has width 0.

### More fun with DNA: Restriction Digest Assays
To perform certain assays, molecular biologists subject DNA sequences to enzymes known as restriction enzymes. There are several types; this is about Type II restriction endonucleases, to be precise. They are usually named with three letters, for the species of origin, and a Roman numeral; e.g., AfeI comes from Alcaligenes faecalis.  These enzymes typically recognize a specific sequence of 6-10 letters, and cut the DNA somewhere in the middle of that sequence.  For example, BgIII recognizes **AGATCT** and cuts between the first **A** and the **G**.

For a typical assay, the DNA will be digested by a "cocktail" of 3-6 enzymes. The lengths of the resulting pieces will be measured by gel electrophoresis.  The lengths should match up with the lengths predicted by an in silico digestion. If not, something is wrong.
Our task is to do the in silico digestion.

For development, here is a dictionary of 4 enzymes, a DNA sequence to digest, and the string we would like to get out of the process. In real life, the DNA sequence would be thousands to ten-thousands of letters long. The researcher could be interested in knowing the cut-points for dozens of enzymes, even though a typical assay uses just a few.

In [62]:

enzymes = {'A-GATCT': 'BgIII',
           'AGC-GCT': 'AfeI',
           'AGG-CCT': 'StuI',
           'AT-CGAT': 'ClaI'}

dna = 'AAAAGCGCTAAAATCGATAAAAAAGATCTAAAAAGCGCT'

goal = 'AAAAGC <AfeI> GCTAAAAT <ClaI> CGATAAAAAA <BgIII> GATCTAAAAAGC <AfeI> GCT'


We are going to use positive look-aheads and look-behinds.  We will build a look ahead-look behind combination for each enzyme.

In [63]:

pats = ['(?<=' + fore + ')(?=' + aft + ')' 
        
        for (fore,aft) in [split(r'-',s) for s in enzymes.keys()]]

pats


['(?<=AT)(?=CGAT)', '(?<=A)(?=GATCT)', '(?<=AGC)(?=GCT)', '(?<=AGG)(?=CCT)']

In [64]:

pats = ' | '.join(pats)
pats


'(?<=AT)(?=CGAT) | (?<=A)(?=GATCT) | (?<=AGC)(?=GCT) | (?<=AGG)(?=CCT)'

In [65]:

pat = "(?x) ( " + pats + " )"
pat


'(?x) ( (?<=AT)(?=CGAT) | (?<=A)(?=GATCT) | (?<=AGC)(?=GCT) | (?<=AGG)(?=CCT) )'

In [66]:
print(goal)

split(pat,dna)


AAAAGC <AfeI> GCTAAAAT <ClaI> CGATAAAAAA <BgIII> GATCTAAAAAGC <AfeI> GCT


['AAAAGC', '', 'GCTAAAAT', '', 'CGATAAAAAA', '', 'GATCTAAAAAGC', '', 'GCT']

It's a good start.  We split in the right places, but we didn't capture the recognition sequences, so we can't retrieve the name of the enzyme from the dictionary. In fact, we captured empty strings.  That's because we captured a zero-width assertion. So we will add some parentheses to capture the look-aheads and look-behinds.

In [67]:

pats = ['(?<=(' + fore + ')) (?=(' + aft + '))' 
        
        for (fore,aft) in [split(r'-',s) for s in enzymes.keys()]]

pats = '  |  '.join(pats)
pat = '(?x) (?: ' + pats + ' )'
pat


'(?x) (?: (?<=(AT)) (?=(CGAT))  |  (?<=(A)) (?=(GATCT))  |  (?<=(AGC)) (?=(GCT))  |  (?<=(AGG)) (?=(CCT)) )'

In [68]:

split(pat,dna)


['AAAAGC',
 None,
 None,
 None,
 None,
 'AGC',
 'GCT',
 None,
 None,
 'GCTAAAAT',
 'AT',
 'CGAT',
 None,
 None,
 None,
 None,
 None,
 None,
 'CGATAAAAAA',
 None,
 None,
 'A',
 'GATCT',
 None,
 None,
 None,
 None,
 'GATCTAAAAAGC',
 None,
 None,
 None,
 None,
 'AGC',
 'GCT',
 None,
 None,
 'GCT']

What happened?  The pattern has eight sets of capturing parentheses. So, the match also returns eight groups when it's executed.  Only the parentheses from the successful alternative will capture anything.  The other six groups are set to **None**.

Happily, **regex** provides a new "branch reset" feature. Briefly, it means that capturing occurs only on the successful branch.

In [69]:

pat = '(?x) (?| ' + pats + ' )'
pat


'(?x) (?| (?<=(AT)) (?=(CGAT))  |  (?<=(A)) (?=(GATCT))  |  (?<=(AGC)) (?=(GCT))  |  (?<=(AGG)) (?=(CCT)) )'

In [70]:

split(pat,dna)


['AAAAGC',
 'AGC',
 'GCT',
 'GCTAAAAT',
 'AT',
 'CGAT',
 'CGATAAAAAA',
 'A',
 'GATCT',
 'GATCTAAAAAGC',
 'AGC',
 'GCT',
 'GCT']

Hooray!  Now all we have to do it to map the recognition sequences into enzyme names:

In [71]:

L = split(pat,dna)

LL = [ ' <' + enzymes[L[i]+'-'+L[i+1]] + '> '+ L[i+2] 
      
                  for i in range(1,len(L),3) ]

L[0] + ''.join(LL)


'AAAAGC <AfeI> GCTAAAAT <ClaI> CGATAAAAAA <BgIII> GATCTAAAAAGC <AfeI> GCT'

Then we can pull it all together into a nice class:

In [72]:

import regex as re

class EndonucleaseDigestor:
    
    def __init__(this,enzymeDict):
        pats = ['(?<=(' + fore + '))(?=(' + aft + '))' 
                for (fore,aft) in [re.split(r'-',s) for s in enzymeDict.keys()]]
        pat = ' | '.join(pats)
        pat = '(?x) (?| ' + pat + ' )'
        this.pat = re.compile(pat)
        this.enzymes = enzymeDict
        
    def digest(this,dna):
        L = this.pat.split(dna)
        LL = [ ' <' + enzymes[L[i]+'-'+L[i+1]] + '> '+ L[i+2] for i in range(1,len(L),3) ]
        return L[0] + ''.join(LL)
        

enzymes = {'A-GATCT': 'BgIII',
           'AGC-GCT': 'AfeI',
           'AGG-CCT': 'StuI',
           'AT-CGAT': 'ClaI'}
dna = 'AAAAGCGCTAAAATCGATAAAAAAGATCTAAAAAGCGCT'
goal = 'AAAAGC <AfeI> GCTAAAAT <ClaI> CGATAAAAAA <BgIII> GATCTAAAAAGC <AfeI> GCT'

digestor = EndonucleaseDigestor(enzymes)
if digestor.digest(dna) == goal: print("passed")


passed


### TMTOWTDT: **sub** with a function
There's another way to solve the restriction digest problem. This time let's start by building a dictionary that maps the recognition sequences into versions with the enzyme name interposed:

In [73]:

d = { sub('-','',k) : sub('-'," <"+v+"> ", k) for k,v in enzymes.items()}
d


{'AGATCT': 'A <BgIII> GATCT',
 'AGCGCT': 'AGC <AfeI> GCT',
 'AGGCCT': 'AGG <StuI> CCT',
 'ATCGAT': 'AT <ClaI> CGAT'}

And let's build a pattern that matches all the recognition sequences:

In [74]:

p = ' | '.join( [ sub('-','',k) for k in enzymes.keys()])
p = '(?x) (' + p + ')'
p


'(?x) (ATCGAT | AGATCT | AGCGCT | AGGCCT)'

The second argument to **sub** can be a function rather than a string.  If so, the function is called with a **match** object as its argument.  We are interested in the first (and only) thing captured in the match and we want to get the corresponding string out of dictionary **d**.  So we define a function to do that.  Then we call sub with that function.

In [75]:

def subber (m):
    print(m)
    return d[m.group(1)]

sub(p, subber, dna)


<regex.Match object; span=(3, 9), match='AGCGCT'>
<regex.Match object; span=(12, 18), match='ATCGAT'>
<regex.Match object; span=(23, 29), match='AGATCT'>
<regex.Match object; span=(33, 39), match='AGCGCT'>


'AAAAGC <AfeI> GCTAAAAT <ClaI> CGATAAAAAA <BgIII> GATCTAAAAAGC <AfeI> GCT'

So we can make a class like this:

In [76]:

import regex as re

class AnotherEndonucleaseDigestor:
    
    def __init__(this,enzymeDict):
        this.d = { re.sub('-','',k) : re.sub('-'," <"+v+"> ", k) for k,v in enzymes.items()}
        p = ' | '.join( [ re.sub('-','',k) for k in enzymes.keys()])
        p = '(?x) (' + p + ')'
        this.pat = re.compile(p)
        this.enzymes = enzymeDict
        
    def digest(this,dna):
        return this.pat.sub( lambda m: this.d[m.group(1)]  , dna)

enzymes = {'A-GATCT': 'BgIII',
           'AGC-GCT': 'AfeI',
           'AGG-CCT': 'StuI',
           'AT-CGAT': 'ClaI'}
dna = 'AAAAGCGCTAAAATCGATAAAAAAGATCTAAAAAGCGCT'
goal = 'AAAAGC <AfeI> GCTAAAAT <ClaI> CGATAAAAAA <BgIII> GATCTAAAAAGC <AfeI> GCT'

digestor = AnotherEndonucleaseDigestor(enzymes)
if digestor.digest(dna) == goal: print("passed")


passed


That probably seems a lot simpler, but there is one problem.  What if two recognition sites are overlapping?  For the in silico simulation of a real digest, it doesn't much matter, because the resolution of gel electrophoresis is much less than the 5-10 bases that might be overlapping.  On the other hand, if the scientist actually wants a complete inventory of all the restriction sites for a large set of enzymes, overlaps matter, and this solution won't work.

### Nested sets
We have seen some character sets such as **[aeiou]** for all (lower-case) vowels.  Suppose we want all lower-case consonants.  One obvious way is to list them all.  We might also be tempted to use set negation:

In [77]:

findall(r'[^aeiou]+',t)


['Th',
 ' q',
 'ck br',
 'wn f',
 'x b',
 'rn ',
 'n 1/23/2013 j',
 'mp',
 'd ',
 'v',
 'r th',
 ' l',
 'zy d',
 'g b',
 'rn ',
 'n 10/6/10.']

The problem is that we get not just consonants, but spaces, digits, etc.
The new **regex** module allows us to do arithmetic on sets:


In [78]:

findall(r'(?x) [[a-z]--[aeiou]]+', t)


['h',
 'q',
 'ck',
 'br',
 'wn',
 'f',
 'x',
 'b',
 'rn',
 'n',
 'j',
 'mp',
 'd',
 'v',
 'r',
 'th',
 'l',
 'zy',
 'd',
 'g',
 'b',
 'rn',
 'n']

### Fuzzy matching
With **regex**, you can specify that patterns need only be satisfied approximately.  You can specify the number of insertions (**i**), number of deletions (**d**), and number of substitutions (**s**) as well as total number of errors (**e**).
This example allows at most one insertion and at most one deletion for each pattern.

In [79]:

list(finditer(r'(brown|lazy){i<=1,d<=1} (dog|fox){i<=1,d<=1}',
              
              'The quick crown fax barn on Monday jumped over the sleazy hog bran on Tuesday.'))


[<regex.Match object; span=(10, 19), match='crown fax', fuzzy_counts=(0, 2, 2)>,
 <regex.Match object; span=(52, 61), match='leazy hog', fuzzy_counts=(0, 2, 1)>]

You can see that the match object reports the number of insertions, deletions, and substitutions as **fuzzy_counts**. 

You can even **require** a minimum number of errors:

In [80]:

list(finditer(r'(brown|lazy){1<=e<=3} (dog|fox){1<=e<=2}',
                            
              'The quick crown fax barn on Monday jumped over the sleazy hog bran on Tuesday.'))


[<regex.Match object; span=(8, 19), match='k crown fax', fuzzy_counts=(2, 2, 0)>,
 <regex.Match object; span=(20, 27), match='barn on', fuzzy_counts=(3, 0, 2)>,
 <regex.Match object; span=(50, 61), match=' sleazy hog', fuzzy_counts=(1, 3, 0)>,
 <regex.Match object; span=(62, 69), match='bran on', fuzzy_counts=(3, 0, 2)>]

What matched what?  We can find out by doing some more capturing.

In [81]:

findall(r'(?:(brown)|(lazy)){1<=e<=3} (?:(dog)|(fox)){1<=e<=2}',
        
        'The quick crown fax barn on Monday jumped over the sleazy hog bran on Tuesday.')
        

[('k crown', '', '', 'fax'),
 ('barn', '', 'on', ''),
 ('', ' sleazy', 'hog', ''),
 ('bran', '', 'on', '')]

What if we try our orginal correct string? We should get back no matches, because there are no errors, right?  Maybe not.

In [82]:

findall(r'(?:(brown)|(lazy)){1<=e<=3} (?:(dog)|(fox)){1<=e<=2}',
        
        'The quick brown fox born on 1/23/2013 jumped over the lazy dog born on 10/6/10.')
 

[('ck brown', '', 'fox', ''),
 (' born', '', 'on', ''),
 ('', 'he lazy', 'dog', ''),
 ('born', '', 'on', '')]

Maybe it will work if we try the **BESTMATCH** option:

In [84]:

findall(r'(brown|lazy){1<=e<=3} (dog|fox){1<=e<=2}',
        'The quick brown fox born on 1/23/2013 jumped over the lazy dog born on 10/6/10.',
        BESTMATCH)


[(' lazy', 'dog '), ('born', 'on')]

Hmm.  Maybe we need the **ENHANCEMATCH** option.

In [85]:

findall(r'(brown|lazy){1<=e<=3} (dog|fox){1<=e<=2}',
        'The quick brown fox born on 1/23/2013 jumped over the lazy dog born on 10/6/10.',
         ENHANCEMATCH)


[(' brown', 'fox'), ('born', 'on'), (' lazy', 'dog '), ('born', 'on')]

Maybe we should use both....

In [86]:

findall(r'(brown|lazy){1<=e<=3} (dog|fox){1<=e<=2}',
        'The quick brown fox born on 1/23/2013 jumped over the lazy dog born on 10/6/10.',
         ENHANCEMATCH | BESTMATCH)


[(' lazy', 'dog '), ('born', 'on')]

OK, now let's try a spelling corrector.  Here's a list of words.

In [87]:
f = open('words.txt')

f.readline()


'Aarhus\n'

In [88]:
words = f.readlines()
words[:10]

['Aaron\n',
 'Ababa\n',
 'aback\n',
 'abaft\n',
 'abandon\n',
 'abandoned\n',
 'abandoning\n',
 'abandonment\n',
 'abandons\n',
 'abase\n']

In [89]:
words =  ' '.join( [sub('\n', '', w) for w in words] )
words[-60:]

'zoom zooms zoos Zorn Zoroaster Zoroastrian Zulu Zulus Zurich'

Now let's make a string with some misspelled (and correct) words. It might seem counterintuitive, but we will take the misspelled words and turn them into a pattern, and use the dictionary as the target sequence.

In [90]:

misspelt = 'abrogatting baandoned abreviat astracted absinthe abussed abus zoan'

misspelt = split('\W+', misspelt)

misspelt = [r"(" + s + r"){e<=2}" for s in misspelt]

misspelt = r"(?x) \m (?: " + " | ".join(misspelt) + r" ) \M"

misspelt


'(?x) \\m (?: (abrogatting){e<=2} | (baandoned){e<=2} | (abreviat){e<=2} | (astracted){e<=2} | (absinthe){e<=2} | (abussed){e<=2} | (abus){e<=2} | (zoan){e<=2} ) \\M'

Note that this time we did not use the brach reset feature.  That's because the captured empty strings are going to tell us which misspelled word was matched.

In [91]:

lis = findall(misspelt,words, ENHANCEMATCH)

lis


[('', 'abandoned', '', '', '', '', '', ''),
 ('', '', '', '', '', '', 'abase', ''),
 ('', '', '', '', '', 'abased', '', ''),
 ('', '', '', '', '', '', 'abash', ''),
 ('', '', '', '', '', 'abashed', '', ''),
 ('', '', '', '', '', '', 'abbe', ''),
 ('', '', 'abbreviate', '', '', '', '', ''),
 ('', '', '', '', '', '', 'abed', ''),
 ('', '', '', '', '', '', 'abet', ''),
 ('', '', '', '', '', '', 'abets', ''),
 ('', '', '', '', '', '', 'able', ''),
 ('', '', '', '', '', '', 'ably', ''),
 ('', '', '', '', '', '', 'Abos', ''),
 ('', '', '', '', '', '', 'about', ''),
 ('abrogating', '', '', '', '', '', '', ''),
 ('', '', '', '', 'absentee', '', '', ''),
 ('', '', '', '', 'absinthe', '', '', ''),
 ('', '', '', 'abstracted', '', '', '', ''),
 ('', '', '', '', '', '', 'Abu', ''),
 ('', '', '', '', '', '', 'abuse', ''),
 ('', '', '', '', '', 'abused', '', ''),
 ('', '', '', '', '', 'abuses', '', ''),
 ('', '', '', '', '', '', 'abut', ''),
 ('', '', '', '', '', '', 'abuts', ''),
 ('', '', '', '', '

Now we will transpose the matrix.  Every column will contain matches for a single misspelled word.  Most of the entries will be empty strings.

In [92]:

z = list(zip(*lis))

z


[('',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  'abrogating',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  'arrogating',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  '',
  

And now we can filter out the empty strings.

In [93]:

z = [ list(filter(lambda s: s!= '', L)) for L in z]

z


[['abrogating', 'arrogating'],
 ['abandoned'],
 ['abbreviate'],
 ['abstracted', 'attracted', 'distracted', 'extracted', 'retracted'],
 ['absentee', 'absinthe'],
 ['abased',
  'abashed',
  'abused',
  'abuses',
  'abutted',
  'abysses',
  'amassed',
  'ambushed',
  'amused',
  'bossed',
  'bused',
  'busied',
  'bussed',
  'busses',
  'busted'],
 ['abase',
  'abash',
  'abbe',
  'abed',
  'abet',
  'abets',
  'able',
  'ably',
  'Abos',
  'about',
  'Abu',
  'abuse',
  'abut',
  'abuts',
  'abyss',
  'aces',
  'adds',
  'ads',
  'ages',
  'ague',
  'aids',
  'aims',
  'airs',
  'alas',
  'album',
  'albums',
  'alms',
  'alum',
  'ambush',
  'amuse',
  'ants',
  'anus',
  'apes',
  'aqua',
  'arcs',
  'arms',
  'arts',
  'as',
  'asks',
  'ass',
  'awls',
  'axes',
  'axis',
  'ayes',
  'babes',
  'Babul',
  'bud',
  'buds',
  'bug',
  'bugs',
  'bum',
  'bums',
  'bun',
  'buns',
  'bus',
  'bush',
  'buss',
  'bust',
  'busy',
  'but',
  'buy',
  'buys',
  'cabs',
  'deus',
  'ebbs',


It's not entirely satisfactory, but it might work well for correcting the spelling of small sets of words, for example, state names.

### Multiple captures
It's now possible to obtain information on all the successful matches of a repeated capture group, not just the last one.  Use **captures** instead of **group**.

In [94]:

dna = 'cgcgcATGcgcattcgggcgTGAcgcgcgTAGcgcgcgcgc'
dna = dna.lower()

orfpat = r'(?x) (?= ( atg  ( (?!tga|tag|taa) ... )*  (?:tga|tag|taa) ))'

search(orfpat,dna).captures(2)


['cgc', 'att', 'cgg', 'gcg']

In [95]:

search(orfpat,dna).captures(1)


['atgcgcattcgggcgtga']

We can also capture things by name.  The string **s** is an excerpt of a long file describing a gene network.  Each line contains two gene names, and the strength of the connection between them.  In this example, we are only interested in gathering the gene names.

In [96]:

s = """AT1G01280	AT1G01450	5.1E-3
AT1G01480	AT1G01560	2.3E-2
AT1G01600	AT1G01610	1.6E-2
AT1G01430	AT1G01630	2.1E-2
AT1G01150	AT1G01700	1.1E-2
"""

m = match(
    r'(?x) (?: (?P<geneA>\w+) \s+ (?P<geneB>\w+) \s+ \S+ \n )*',
    s)

m.capturesdict()


{'geneA': ['AT1G01280', 'AT1G01480', 'AT1G01600', 'AT1G01430', 'AT1G01150'],
 'geneB': ['AT1G01450', 'AT1G01560', 'AT1G01610', 'AT1G01630', 'AT1G01700']}

We can even reuse a name;

In [97]:

m = match(
    r'(?x) (?: (?P<gene>\w+) \s+ (?P<gene>\w+) \s+ \S+ \n )*',
    s)

m.capturesdict()


{'gene': ['AT1G01280',
  'AT1G01450',
  'AT1G01480',
  'AT1G01560',
  'AT1G01600',
  'AT1G01610',
  'AT1G01430',
  'AT1G01630',
  'AT1G01150',
  'AT1G01700']}

In [98]:
sorted(set(m.capturesdict()['gene']))

['AT1G01150',
 'AT1G01280',
 'AT1G01430',
 'AT1G01450',
 'AT1G01480',
 'AT1G01560',
 'AT1G01600',
 'AT1G01610',
 'AT1G01630',
 'AT1G01700']

### Reverse searching

Searches can now work backwards:

Note: the result of a reverse search is not necessarily the reverse of a forward search:

In [99]:

findall(r"(?r)..", "abcde")


['de', 'bc']

Who cares?  I thought of an example.

In [100]:
sub(r'(?rx) (\d\d\d)',
    r',\1',
    '1 mile = 1760 yards = 5280 ft = 63360 in = 1609344 mm = 160934.4 cm, more or less.  Pi = 3.14159')

'1 mile = 1,760 yards = 5,280 ft = 63,360 in = 1,609,344 mm = ,160,934.4 cm, more or less.  Pi = 3.14,159'

In [101]:
sub(r'(?rx)  (?<=\d) (\d\d\d)',
    r',\1',
    '1 mile = 1760 yards = 5280 ft = 63360 in = 1609344 mm = 160934.4 cm, more or less.  Pi = 3.14159')

'1 mile = 1,760 yards = 5,280 ft = 63,360 in = 1,609,344 mm = 160,934.4 cm, more or less.  Pi = 3.14,159'

In [102]:
sub(r'(?rx)  (?<! [.] \d*) (?<=\d) (\d\d\d)',
    r',\1',
    '1 mile = 1760 yards = 5280 ft = 63360 in = 1609344 mm = 160934.4 cm, more or less.  Pi = 3.14159')

'1 mile = 1,760 yards = 5,280 ft = 63,360 in = 1,609,344 mm = 160,934.4 cm, more or less.  Pi = 3.14159'

### POSIX Matching (Leftmost Longest)

The default matching method for alternations is to match the first alternative that will match. The POSIX standard is to find the leftmost longest match. This can be turned on using the POSIX flag **(?p)**.

In [103]:

list(finditer( r'(dog|doge|doggerel)', 'The doge wrote nothing but doggerel.'))


[<regex.Match object; span=(4, 7), match='dog'>,
 <regex.Match object; span=(27, 30), match='dog'>]

In [104]:

list(finditer( r'(?p)(dog|doge|doggerel)', 'The doge wrote nothing but doggerel.'))


[<regex.Match object; span=(4, 8), match='doge'>,
 <regex.Match object; span=(27, 35), match='doggerel'>]


### **fullmatch**
The pattern must match the entire string.

In [105]:

match(r'The doge', 'The doge wrote nothing but doggerel.')


<regex.Match object; span=(0, 8), match='The doge'>

In [106]:

fullmatch(r'The doge', 'The doge wrote nothing but doggerel.')


Nope, that one didn't match.

### Partial matching
Can the target string be extended to match the pattern?  The optional **partial** argument to **match**, **search**, and **fullmatch** can answer this question. This could be useful if you are validating input from the terminal, for example.

This one is true, because the target can be extended to 'The doge wrote nothing but doggerel.' to match the pattern.  But, if you think about it, you will see that this one is true for any target string.  (It can be extended with 'dogdog'.)

In [107]:

fullmatch(r'.*dog.*dog.*', 'The doge wrote nothing', partial=True)


<regex.Match object; span=(0, 22), match='The doge wrote nothing', partial=True>

This one is more interesting: Can the string be extended to be a Social Security Number?

In [108]:

fullmatch(r'\d\d\d-\d\d-\d\d\d\d',  "999-89-7", partial=True)


<regex.Match object; span=(0, 8), match='999-89-7', partial=True>

In [109]:

match(r'\d\d\d-\d\d-\d\d\d\d',  "999-89-7", partial=True)


<regex.Match object; span=(0, 8), match='999-89-7', partial=True>

In [110]:

fullmatch(r'\d\d\d-\d\d-\d\d\d\d',  "My SSN is 999-89-7", partial=True)


In [111]:

match(r'\d\d\d-\d\d-\d\d\d\d',  "My SSN is 999-89-7", partial=True)


In [112]:

search(r'\d\d\d-\d\d-\d\d\d\d',  "My SSN is 999-89-7", partial=True)


<regex.Match object; span=(10, 18), match='999-89-7', partial=True>

Notice that this one is a complete match, so the **partial** field is missing from the match object:

In [113]:

search(r'\d\d\d-\d\d-\d\d\d\d',  "My SSN is 999-89-7654, but don't tell.", partial=True)


<regex.Match object; span=(10, 21), match='999-89-7654'>

In [114]:

search(r'\d\d\d-\d\d-\d\d\d\d',  "My SSN is 999-89-76, but don't tell.", partial=True)


<regex.Match object; span=(33, 36), match='ll.', partial=True>

### Some functional programming fun

In [115]:

def twice(f):
    return lambda x: f(f(x))

def prepender(s):
    return lambda t: s + t

twice(twice)(twice(prepender('spam ')))('eggs and spam')


'spam spam spam spam spam spam spam spam eggs and spam'

In [116]:

twice(twice)(twice(prepender(len('spam '))))(len('eggs and spam'))


53

### Mark's puzzle
Which character is most frequent in a string?

In [122]:
def most(s, care_about=r'\w'):
    t=''.join(sorted(s))
    p = r'((' + care_about + r')\2*)'
    L = [ m.group(1) for m in finditer(p, t) ]
    m = max(L, key=len)
    return (m[0], len(m))

most('123462232340997092')

('2', 5)

In [118]:
most('123462232340997092', care_about='[13579]')

('3', 3)

In [123]:
most(twice(twice)(twice(prepender('spam ')))('eggs and spam'))

('a', 10)

In [124]:
most(twice(twice)(twice(prepender('spam ')))('eggs and spam'), r'[^aeiou\s]')

('s', 10)